# **DVD Rental Data Project**

## **Project Overview**

This project was created to equip our DVD Rental company with a set of tools that will facilitate data-driven decision-making and support the **growth and success of our operations**.

The project is divided into **three stages**:

### **Stage 1: Database Creation**
We will create a SQLite database named **dvdrental.db** and populate it with the company’s historical data, which is currently stored in multiple CSV files. These files, compiled in **csv_zip.zip**, have served as a temporary data storage solution.

### **Stage 2: Data Analysis**
Using Pandas, we will perform an in-depth analysis to extract actionable insights and uncover patterns that can help optimize our processes and better understand our customer base.

### **Visualization & Dashboard**
Finally, we will build an interactive Power BI dashboard to provide the management team with near-real-time insights, enabling more informed and timely decisions.

---

## **Stage 1: Database Creation**
- Create a SQLite database named **`dvdrental.db`**.  
- Populate it with the company’s historical data, which is currently stored in multiple CSV files.  
- These files, compiled in **`csv_zip.zip`**, have been used as a temporary data storage solution.

We are going to use Python and the sqlite3 library to set up the database, zipfile to extract the CSV files and Pandas to preview each file before adding constraints.

---

In [4]:
import sqlite3 as sql
import pandas as pd
import glob as glob
import os
import zipfile

In [5]:
# dest_folder = '\\Databases'

In [6]:
db_name = "dvdrental.db"

In [62]:
origin = "csv_zip.zip"

### Database creation

This script creates the .db file (database file), extracts each file from the zip file with a loop, reads it and saves it in the database with Pandas, and closes the connection.

In [63]:
#with sql.connect(os.path.join(dest_folder, db_name)) as conn:
with sql.connect(db_name) as conn:
    # df_dict = {}
    with zipfile.ZipFile(origin) as z:
        for file_name in z.namelist():
            if file_name.endswith('.csv'):
                table = os.path.splitext(file_name)[0]
                with z.open(file_name) as f:
                    df = pd.read_csv(f)
                    df.to_sql(table, conn, if_exists='replace', index=False)
                    # df_dict[os.path.splitext(file_name)[0]] = pd.read_csv(f)
                    print(f'Loaded {file_name} into table {table}')

Loaded country.csv into table country
Loaded film_category.csv into table film_category
Loaded store.csv into table store
Loaded staff.csv into table staff
Loaded film.csv into table film
Loaded city.csv into table city
Loaded address.csv into table address
Loaded customer.csv into table customer
Loaded inventory.csv into table inventory
Loaded category.csv into table category
Loaded actor.csv into table actor
Loaded language.csv into table language
Loaded film_actor.csv into table film_actor
Loaded rental.csv into table rental


In [66]:
# db = os.path.join(dest_folder, db_name)

### Confirming tables

Here we check all the tables that now exist in the .db file

In [66]:
with sql.connect(db_name) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
    tables = cursor.fetchall()

    print("Tables in db:")

    for table in tables:
        print("-", table[0])

Tables in db:
- country_old
- film_category_old
- store_old
- staff_old
- film_old
- city_old
- address_old
- customer_old
- inventory_old
- category_old
- actor_old
- language_old
- film_actor_old
- rental_old


### Query Function

Now we define the query that will be used to read and write to the database. It handles SELECT statements with cursor.fetchall(), and statements that write to the database with conn.commit().

In SQLite, foreign keys are disabled by default, so we pass the conn.execute("PRAGMA foreign_keys = ON;") instruction so that they are enabled for every connection.

In [83]:
def query_func(dbase: str, query:"str", params: tuple | list | None =None):
    try:
        with sql.connect(dbase) as conn:
            
            conn.execute("PRAGMA foreign_keys = ON;")
            cursor = conn.cursor()
            cursor.execute(query, params or ())

            # query_type = query.strip().split()[0].upper()
            # print(f"Running a {query_type} query...")
    
            # Define behavior for SELECT queries
            # if query.strip().upper().startswith("SELECT"):
            if cursor.description:
                # Get column names
                columns = [desc[0] for desc in cursor.description]
        
                # Fetch data
                data = cursor.fetchall()
        
                # Return DataFrame        
                return pd.DataFrame(data, columns=columns)
            # For write operations
            conn.commit()
            return cursor.rowcount
 
    except sql.Error as e:
        print(f"SQLite Error: {e}")
        raise
        

### Adding Constraints

To add constraints we need to follow a set of steps that are necessary when working with SQLite3. This is because in SQLite we can only rename tables and add new columns but won't allow constraints to be added to existing tables:

1. Use ALTER TABLE to change the name of a table
2. CREATE TABLE to create the new table with the original name and add the constraints
3. INSERT INTO and SELECT to move all rows from the old tables to the new ones
4. DROP TABLE to remove the previous versions of the tables. 

#### 1. Use ALTER TABLE to change the name of a table

Now we need to change the names of the existing tables to later replace them with the constrained versions. We also need to load the tables into dataframes so we can inspect them and know what constraints need to be applied.

We acomplish that here, first cleaning the names and then using a loop to query each table, load them into dataframes and store them inside a dictionary. Finally, we loop through each table in the dictionary to use an ALTER TABLE query that ultimately renames all tables in the database.

In [33]:
tables_clean = []
for table in tables:
    tables_clean.append(table[0])

tables_clean

['country',
 'film_category',
 'store',
 'staff',
 'film',
 'city',
 'address',
 'customer',
 'inventory',
 'category',
 'actor',
 'language',
 'film_actor',
 'rental']

In [75]:
df_dict = {}

for table in tables_clean:
    df_dict[table] = query_func(db_name, f"SELECT * FROM {table};")
    print(f"{table} was successfully added to df_dict")

Running a SELECT query...
country was successfully added to df_dict
Running a SELECT query...
film_category was successfully added to df_dict
Running a SELECT query...
store was successfully added to df_dict
Running a SELECT query...
staff was successfully added to df_dict
Running a SELECT query...
film was successfully added to df_dict
Running a SELECT query...
city was successfully added to df_dict
Running a SELECT query...
address was successfully added to df_dict
Running a SELECT query...
customer was successfully added to df_dict
Running a SELECT query...
inventory was successfully added to df_dict
Running a SELECT query...
category was successfully added to df_dict
Running a SELECT query...
actor was successfully added to df_dict
Running a SELECT query...
language was successfully added to df_dict
Running a SELECT query...
film_actor was successfully added to df_dict
Running a SELECT query...
rental was successfully added to df_dict


In [76]:
df_dict.keys()

dict_keys(['country', 'film_category', 'store', 'staff', 'film', 'city', 'address', 'customer', 'inventory', 'category', 'actor', 'language', 'film_actor', 'rental'])

In [65]:
for table in tables_clean:
    query_func(db_name, f"ALTER TABLE {table} RENAME TO {table}_old;")

Running a ALTER query...
Running a ALTER query...
Running a ALTER query...
Running a ALTER query...
Running a ALTER query...
Running a ALTER query...
Running a ALTER query...
Running a ALTER query...
Running a ALTER query...
Running a ALTER query...
Running a ALTER query...
Running a ALTER query...
Running a ALTER query...
Running a ALTER query...


#### 2. CREATE TABLE to create the new table with the original name and add the constraints

Here we start using Pandas to preview each table, check if it contains null values and then use the query function to create the new version of each table.

In [64]:
df_dict["country"].head()

,country_id,country,last_update
0,1,Afghanistan,2006-02-15 09:44:00
1,2,Algeria,2006-02-15 09:44:00
2,3,American Samoa,2006-02-15 09:44:00
3,4,Angola,2006-02-15 09:44:00
4,5,Anguilla,2006-02-15 09:44:00


In [77]:
df_dict["country"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   country_id   109 non-null    int64 
 1   country      109 non-null    object
 2   last_update  109 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.7+ KB


In [68]:
query_func(
    db_name,

   """
   CREATE TABLE country (
       country_id INTEGER PRIMARY KEY,
       country TEXT NOT NULL,
       last_update DATETIME DEFAULT CURRENT_TIMESTAMP
   )

   """
           
)

Running a CREATE query...


-1

In [69]:
df_dict['actor'].head()

,actor_id,first_name,last_name,last_update
0,1,Penelope,Guiness,2013-05-26 14:47:57.62
1,2,Nick,Wahlberg,2013-05-26 14:47:57.62
2,3,Ed,Chase,2013-05-26 14:47:57.62
3,4,Jennifer,Davis,2013-05-26 14:47:57.62
4,5,Johnny,Lollobrigida,2013-05-26 14:47:57.62


In [78]:
df_dict['actor'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   actor_id     200 non-null    int64 
 1   first_name   200 non-null    object
 2   last_name    200 non-null    object
 3   last_update  200 non-null    object
dtypes: int64(1), object(3)
memory usage: 6.4+ KB


In [72]:
query_func(
    db_name,

   """
    
   
   CREATE TABLE actor (
       actor_id INTEGER PRIMARY KEY,
       first_name TEXT NOT NULL,
       last_name TEXT NOT NULL,
       last_update DATETIME DEFAULT CURRENT_TIMESTAMP
   )

   """
           
)

Running a CREATE query...


-1

In [80]:
df_dict['city'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   city_id      600 non-null    int64 
 1   city         600 non-null    object
 2   country_id   600 non-null    int64 
 3   last_update  600 non-null    object
dtypes: int64(2), object(2)
memory usage: 18.9+ KB


In [82]:
df_dict['city'].head()

,city_id,city,country_id,last_update
0,1,A Corua (La Corua),87,2006-02-15 09:45:25
1,2,Abha,82,2006-02-15 09:45:25
2,3,Abu Dhabi,101,2006-02-15 09:45:25
3,4,Acua,60,2006-02-15 09:45:25
4,5,Adana,97,2006-02-15 09:45:25


In [87]:
query_func(
    db_name,

   """
   CREATE TABLE city (
       city_id INTEGER PRIMARY KEY,
       city TEXT NOT NULL,
       country_id INTEGER NOT NULL,
       last_update DATETIME DEFAULT CURRENT_TIMESTAMP,
       FOREIGN KEY (country_id) REFERENCES country(country_id)
   )

   """
           
)

-1

In [79]:
df_dict['address'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 603 entries, 0 to 602
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   address_id   603 non-null    int64  
 1   address      603 non-null    object 
 2   address2     0 non-null      object 
 3   district     600 non-null    object 
 4   city_id      603 non-null    int64  
 5   postal_code  599 non-null    float64
 6   phone        601 non-null    float64
 7   last_update  603 non-null    object 
dtypes: float64(2), int64(2), object(4)
memory usage: 37.8+ KB


In [74]:
df_dict['address'].head()

,address_id,address,address2,district,city_id,postal_code,phone,last_update
0,1,47 MySakila Drive,None,Alberta,300,NaN,NaN,2006-02-15 09:45:30
1,2,28 MySQL Boulevard,None,QLD,576,NaN,NaN,2006-02-15 09:45:30
2,3,23 Workhaven Lane,None,Alberta,300,NaN,1.403334e+10,2006-02-15 09:45:30
3,4,1411 Lillydale Drive,None,QLD,576,NaN,6.172236e+09,2006-02-15 09:45:30
4,5,1913 Hanoi Way,None,Nagasaki,463,35200.0,2.830338e+10,2006-02-15 09:45:30


In [90]:
query_func(
    db_name,

   """
   CREATE TABLE address (
       address_id INTEGER PRIMARY KEY,
       address TEXT NOT NULL,
       address2 TEXT,
       distric TEXT,
       city_id INTEGER NOT NULL,
       postal_code TEXT,
       phone TEXT,
       last_update DATETIME DEFAULT CURRENT_TIMESTAMP,
       FOREIGN KEY (city_id) REFERENCES city(city_id)
   )

   """
           
)

-1

In [91]:
df_dict['category'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category_id  16 non-null     int64 
 1   name         16 non-null     object
 2   last_update  16 non-null     object
dtypes: int64(1), object(2)
memory usage: 516.0+ bytes


In [92]:
df_dict['category'].head()

,category_id,name,last_update
0,1,Action,2006-02-15 09:46:27
1,2,Animation,2006-02-15 09:46:27
2,3,Children,2006-02-15 09:46:27
3,4,Classics,2006-02-15 09:46:27
4,5,Comedy,2006-02-15 09:46:27


In [94]:
query_func(
    db_name,

   """
   CREATE TABLE category (
       category_id INTEGER PRIMARY KEY,
       name TEXT NOT NULL,
       last_update DATETIME DEFAULT CURRENT_TIMESTAMP
   )

   """
           
)

-1

In [100]:
df_dict['language'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   language_id  6 non-null      int64 
 1   name         6 non-null      object
 2   last_update  6 non-null      object
dtypes: int64(1), object(2)
memory usage: 276.0+ bytes


In [101]:
df_dict['language'].head()

,language_id,name,last_update
0,1,English,2006-02-15 10:02:19
1,2,Italian,2006-02-15 10:02:19
2,3,Japanese,2006-02-15 10:02:19
3,4,Mandarin,2006-02-15 10:02:19
4,5,French,2006-02-15 10:02:19


In [102]:
query_func(
    db_name,

   """
   CREATE TABLE language (
       language_id INTEGER PRIMARY KEY,
       name TEXT NOT NULL,
       last_update DATETIME DEFAULT CURRENT_TIMESTAMP
   );

   """
           
)

-1

In [120]:
chk = df_dict['film'].drop_duplicates('rating')

chk2 = sorted(chk['rating'].values)
chk2

['G', 'NC-17', 'PG', 'PG-13', 'R']

In [109]:
df_dict['film'].head()

,film_id,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext
0,133,Chamber Italian,A Fateful Reflection of a Moose And a Husband ...,2006,1,7,4.99,117,14.99,NC-17,2013-05-26 14:50:58.951,{Trailers},'chamber':1 'fate':4 'husband':11 'italian':2 ...
1,384,Grosse Wonderful,A Epic Drama of a Cat And a Explorer who must ...,2006,1,5,4.99,49,19.99,R,2013-05-26 14:50:58.951,"{""Behind the Scenes""}",'australia':18 'cat':8 'drama':5 'epic':4 'exp...
2,8,Airport Pollock,A Epic Tale of a Moose And a Girl who must Con...,2006,1,6,4.99,54,15.99,R,2013-05-26 14:50:58.951,{Trailers},'airport':1 'ancient':18 'confront':14 'epic':...
3,98,Bright Encounters,A Fateful Yarn of a Lumberjack And a Feminist ...,2006,1,4,4.99,73,12.99,PG-13,2013-05-26 14:50:58.951,{Trailers},'boat':20 'bright':1 'conquer':14 'encount':2 ...
4,1,Academy Dinosaur,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,2013-05-26 14:50:58.951,"{""Deleted Scenes"",""Behind the Scenes""}",'academi':1 'battl':15 'canadian':20 'dinosaur...


In [103]:
df_dict['film'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   film_id           1000 non-null   int64  
 1   title             1000 non-null   object 
 2   description       1000 non-null   object 
 3   release_year      1000 non-null   int64  
 4   language_id       1000 non-null   int64  
 5   rental_duration   1000 non-null   int64  
 6   rental_rate       1000 non-null   float64
 7   length            1000 non-null   int64  
 8   replacement_cost  1000 non-null   float64
 9   rating            1000 non-null   object 
 10  last_update       1000 non-null   object 
 11  special_features  1000 non-null   object 
 12  fulltext          1000 non-null   object 
dtypes: float64(2), int64(5), object(6)
memory usage: 101.7+ KB


In [121]:
query_func(
    db_name,

   """
   CREATE TABLE film (
       film_id INTEGER PRIMARY KEY,
       title TEXT NOT NULL,
       description TEXT NOT NULL,
       release_year INTEGER NOT NULL,
       language_id INTEGER NOT NULL,
       rental_duration INTEGER NOT NULL CHECK (rental_duration > 0),
       rental_rate REAL NOT NULL CHECK (rental_rate > 0),
       length INTEGER NOT NULL CHECK (length > 0),
       replacement_cost REAL NOT NULL CHECK (replacement_cost >= rental_rate),
       rating TEXT NOT NULL CHECK (rating IN ('G', 'NC-17', 'PG', 'PG-13', 'R')),
       last_update DATETIME DEFAULT CURRENT_TIMESTAMP,
       special_features TEXT,
       fulltext TEXT,
       FOREIGN KEY (language_id) REFERENCES language (language_id)
   );

   """
           
)

-1

In [123]:
df_dict['film_actor'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5462 entries, 0 to 5461
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   actor_id     5462 non-null   int64 
 1   film_id      5462 non-null   int64 
 2   last_update  5462 non-null   object
dtypes: int64(2), object(1)
memory usage: 128.1+ KB


In [124]:
query_func(
    db_name,

   """
   CREATE TABLE film_actor (
       actor_id INTEGER PRIMARY KEY,
       film_id INTEGER NOT NULL,
       last_update DATETIME DEFAULT CURRENT_TIMESTAMP,
       FOREIGN KEY (film_id) REFERENCES film(film_id)
   );

   """
           
)

-1

In [125]:
df_dict['film_category'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   film_id      1000 non-null   int64 
 1   category_id  1000 non-null   int64 
 2   last_update  1000 non-null   object
dtypes: int64(2), object(1)
memory usage: 23.6+ KB


In [136]:
query_func(
    db_name,

   """
   CREATE TABLE film_category (
       film_id INTEGER PRIMARY KEY,
       category_id INTEGER NOT NULL,
       last_update DATETIME DEFAULT CURRENT_TIMESTAMP,
       FOREIGN KEY (film_id) REFERENCES film(film_id),
       FOREIGN KEY (category_id) REFERENCES category(category_id)
   );

   """
           
)

-1

In [140]:
df_dict['staff'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   staff_id     2 non-null      int64 
 1   first_name   2 non-null      object
 2   last_name    2 non-null      object
 3   address_id   2 non-null      int64 
 4   email        2 non-null      object
 5   store_id     2 non-null      int64 
 6   active       2 non-null      int64 
 7   username     2 non-null      object
 8   password     2 non-null      object
 9   last_update  2 non-null      object
 10  picture      1 non-null      object
dtypes: int64(4), object(7)
memory usage: 308.0+ bytes


#### 3. INSERT INTO and SELECT to move all rows from the old tables to the new ones

We arrived to the step at which we need to copy all the contents from the old tables to the new ones. This is achieved using a for loop and INSERT INTO each new able and using SELECT * from the old tables. 

We opted not to use the function this time, because we want to do it in one connection all at once and avoid inefficiencies.

In [ ]:
for table in tables_clean:
    query_func(
        db_name,
        
        f"""
        INSERT INTO {table}
        SELECT * FROM {table}_old
        
        """
    )

#### 4. DROP TABLE to remove the previous versions of the tables.

After making sure our new tables are correct, we can then use a for loop to DROP the old versions.